환경 설정

In [65]:
''' Time Sleep '''
# time.sleep(random.uniform(1.3, 5))

In [ ]:
!pip install webdriver-manager
!pip install selenium
!pip install lxml
!pip install requests
!pip install beautifulsoup4 requests
!pip install tqdm

In [ ]:
!pip install requests beautifulsoup4

In [3]:
import requests
from lxml import html
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import pandas as pd
import time
import random

### Nate Pann 크롤링 시작

#### Selenium 으로 Click, Back 반복

In [4]:
# 데이터프레임 초기화
nate_df = pd.DataFrame(columns=['Date', 'URL', 'Title', 'Body Text', 'Replies'])

# Chrome WebDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [7]:
while True:
    try:
        for i in range(49):

            ''' URL 열기 '''
            url = f'https://pann.nate.com/search/talk?q=%EA%B3%B5%EA%B0%95&page={i+1}'
            driver.get(url)
            wait = WebDriverWait(driver, 10)
            print(f'page{i+1}')

            # 페이지가 정상적으로 로드되었는지 확인
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            if response.status_code != 200:
                print("페이지 로드 실패:", response.status_code)
                continue

            ''' 글 Title 을 가져오기 전 준비 '''
            tree = html.fromstring(response.content)
            subject_elements = driver.find_elements(By.CLASS_NAME, 'subject')

            try :
                ''' 글 클릭하기 '''
                for j in range(len(subject_elements)):
                    # 클릭!
                    time.sleep(random.uniform(1.8, 3))
                    subject_elements[j].click()
                    time.sleep(random.uniform(1.8, 2))
                    print(f'{i+1}page / {j+1}번째 글 / click!')

                    new_lst = []

                    ''' Date Check '''
                    time.sleep(random.uniform(1.8, 2.5))
                    post_date = driver.find_element(By.CLASS_NAME, 'date')
                    # 연도 추출
                    year = post_date.text.split('.')[0]  # '2024' 부분 추출
                    two_year = int(year[-2:])     # 마지막 두 자리 추출
                    
                    if two_year > 15 :
                        
                        ''' Date append '''
                        new_lst.append(post_date.text)
                        # print(f'{j}번째 date appending')

                        ''' URL append '''
                        time.sleep(random.uniform(1.3, 2))
                        new_lst.append(driver.current_url)
                        # print(f'{j}번째 URL appending')

                        ''' Title append '''
                        time.sleep(random.uniform(2.1, 3))
                        h1 = driver.find_element(By.TAG_NAME, 'h1')
                        new_lst.append(h1.text)
                        # print(f'{j}번째 title appending')

                        ''' Body Text append '''
                        time.sleep(random.uniform(1.3, 2))
                        body = driver.find_element(By.ID, 'contentArea')
                        new_lst.append(body.text)
                        # print(f'{j}번째 body text appending')

                        ''' Replies append '''
                        replies_lst = []
                        usertxt_elements = driver.find_elements(By.CLASS_NAME, 'usertxt')

                        for usertxt in usertxt_elements:
                            span_text = usertxt.find_element(By.TAG_NAME, 'span').text
                            replies_lst.append(span_text)
                        
                        new_lst.append(replies_lst)

                        ''' Data Frame 에 추가 '''
                        new_df = pd.DataFrame([new_lst], columns=['URL', 'Date', 'Title', 'Body Text', 'Replies'])
                        nate_df = pd.concat([nate_df, new_df], ignore_index=True)
                        print('Data concat success')

                        ''' 뒤로가기 '''
                        driver.back()  # 이전 페이지로 돌아가기
                        print('back!')
                        time.sleep(random.uniform(3, 5.4))  # 페이지가 로드될 시간을 기다림
                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'subject')))  # 페이지 로드 대기
                        subject_elements = driver.find_elements(By.CLASS_NAME, 'subject')  # 다시 요소 찾기

                    else :
                        ''' 뒤로가기 '''
                        driver.back()  # 이전 페이지로 돌아가기
                        print('back!')
                        time.sleep(random.uniform(3, 5.4))  # 페이지가 로드될 시간을 기다림
                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'subject')))  # 페이지 로드 대기
                        subject_elements = driver.find_elements(By.CLASS_NAME, 'subject')  # 다시 요소 찾기

            except Exception as e:
                print(f"오류 발생2 : {e}")
                # 엔터 쳐야하는 곳
                time.sleep(random.uniform(2, 3.5))
                # 엔터 키 입력
                body = driver.find_element(By.TAG_NAME, 'body')  # body 요소 찾기
                body.send_keys(Keys.ENTER)  # 엔터 키 입력

    except Exception as e:
        print(f"오류 발생1 : {e}")
        exit()

# 드라이버 종료
# driver.quit()

page1
1page / 1번째 글 / click!
Data concat success
back!
1page / 2번째 글 / click!


KeyboardInterrupt: 

In [ ]:
# 변환된 데이터를 저장할 리스트
transformed_data = []

# 데이터 변환
for row in nate_df.itertuples(index=False):
    base_values = row[:-1]  # 마지막 요소 제외
    last_item = row[-1]     # 마지막 요소
    if isinstance(last_item, list):  # 마지막 요소가 리스트인지 확인
        for item in last_item:  # 리스트를 반복
            transformed_data.append(base_values + (item,))  # base_values와 item을 합쳐서 추가

# 데이터 프레임 생성
df_nate_pann = pd.DataFrame(transformed_data, columns=['Date', 'URL', 'Title', 'Body Text', 'Replies'])

# 결과 출력
print(df_nate_pann)

In [ ]:
import pickle

# Save data to a pickle file
file_path = r"C:\Users\trainee225\Desktop\LG\이리저리요리조리\nate_pann_공강crawling(1-50page)_draft.pkl"
with open(file_path, 'wb') as file :
    pickle.dump(nate_df, file)

print("Data has been saved to data.pkl")

# TEST CODE

In [60]:
''' test code '''
### concat

# 데이터프레임에 데이터 추가
testdata = ['2024-09-01', '첫 번째 제목', '본문 내용 1', 5]
test_df = pd.DataFrame([testdata], columns=['Date', 'Title', 'Body Text', 'Replies'])

# 추가할 데이터 리스트를 데이터프레임으로 바꾸기
new_data = [['2024-09-02', '두 번째 제목', '본문 내용 2', 10]]
new_df = pd.DataFrame(new_data, columns=['Date', 'Title', 'Body Text', 'Replies'])

# 데이터프레임 합치기
test_df = pd.concat([test_df, new_df], ignore_index=True)

# 결과 출력
test_df

Date    Title Body Text  Replies
0  2024-09-01  첫 번째 제목   본문 내용 1        5
1  2024-09-02  두 번째 제목   본문 내용 2       10

In [38]:
''' test code '''
### append

testlst = []
testlst.append(1)
testlst.append(2)
testlst.append(3)
testlst

[1, 2, 3]

In [17]:
# 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹 페이지 열기
url = 'https://pann.nate.com/search/talk?q=%EA%B3%B5%EA%B0%95&page=1'
driver.get(url)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 10)